In [1]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

import random
from tqdm import tqdm
from utils import get_model

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Set Parameters

In [ ]:
data_dir='./data'
batch_size=64
valid_size=0.1
num_classes = 10
num_epochs = 20
learning_rate = 0.01
val = 4

## Get Dataset and DataLoader

In [ ]:
normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

# define transforms
transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize,
    ])

# load the dataset
train_dataset = datasets.CIFAR10(
    root=data_dir, train=True,
    download=True, transform=transform,
)

val_dataset = datasets.CIFAR10(
    root=data_dir, train=True,
    download=True, transform=transform,
)

test_dataset = datasets.CIFAR10(
    root=data_dir, train=False,
    download=True, transform=transform,
)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))

# if shuffle:
np.random.seed(42)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, sampler=train_sampler)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, sampler=valid_sampler)

test_loader = torch.utils.data.DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False
        )

## Get Model

In [2]:
import ipywidgets as widgets
from IPython.display import display

# List of available CNN models
model_options = [
    "efficientnet_b0",
    "inception_v1",
    "mobilenet_v2",
    "resnet18",
    "resnet50",
    "resNeXt50",
    "vgg19",
]
# Dropdown widget
model_dropdown = widgets.Dropdown(options=model_options, description="Select Model:")
# Display widget
display(model_dropdown)

Dropdown(description='Select Model:', options=('efficientnet_b0', 'inception_v1', 'mobilenet_v2', 'resnet18', …

In [9]:
model = get_model.get_model(model_dropdown.value, num_classes=10)
print(f"{model_dropdown.value} loaded successfully")


inception_v1 loaded successfully


In [8]:
model


GoogleNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (inception3a): Inception_Block(
    (layer1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

## Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

### Send model to device

In [ ]:
model.to(device)

## Train and Validate

In [ ]:
for epoch in tqdm(range(num_epochs)):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, loss.item()))

    # Validation
    if epoch % val == 0 and epoch > 0:
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))